In [4]:
import requests
from bs4 import BeautifulSoup
import csv
import xml.etree.ElementTree as ET
import time
import pandas as pd

from requests.auth import HTTPBasicAuth

In [5]:
#dataframe with the min max and total amount of documents
columns = ['year', 'min', 'max', 'total']
df = pd.DataFrame(columns=columns)

In [6]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

def get_token():
    # Request access token
    response = requests.post(
        token_url,
        auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
        data={"grant_type": "client_credentials"},
    )
    
    # Extract token
    if response.status_code == 200:
        token = response.json()["access_token"]
        return token
    else:
        print("Failed to retrieve access token.")
        return None

In [7]:
def get_amount_of_patents(year, token):

    search_url = "https://ops.epo.org/rest-services/published-data/search"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
    }
    
    early_params = {
    "q": f'pd within "{year},{year}"',
    "range": "1-100",
    "sort": "publicationDate asc"
    }
    
    late_params = {
    "q": f'pd within "{year},{year}"',
    "range": "1-100",
    "sort": "publicationDate desc"
    }
    
    early_response = requests.get(search_url, headers=headers, params=early_params)
    late_response = requests.get(search_url, headers=headers, params=late_params)
    

    if early_response.status_code == 200 and late_response.status_code == 200:
        early_root = ET.fromstring(early_response.content)
        late_root = ET.fromstring(late_response.content)
        
        # Namespace dictionary (important for parsing)
        ns = {"epo": "http://www.epo.org/exchange"}
        
        early_doc_numbers = []
        late_doc_numbers = []
        for doc in early_root.findall(".//epo:document-id", namespaces=ns):
            doc_number = doc.find("epo:doc-number", namespaces=ns)
            if doc_number is not None and doc_number.text.isdigit():
                early_doc_numbers.append(int(doc_number.text))
        
        for doc in late_root.findall(".//epo:document-id", namespaces=ns):
            doc_number = doc.find("epo:doc-number", namespaces=ns)
            if doc_number is not None and doc_number.text.isdigit():
                late_doc_numbers.append(int(doc_number.text))
        
        #min_early = min(int(doc) for doc in early_doc_numbers if 1000 < int(doc) < 25_000_000)
        #max_late  = max(int(doc) for doc in late_doc_numbers  if 1000 < int(doc) < 25_000_000)
        
        if not early_doc_numbers or not late_doc_numbers:
            return None, None
        
        return min(early_doc_numbers), max(late_doc_numbers)
         

In [ ]:
print(get_amount_of_patents(1883, get_token()))

In [11]:
for year in range(1772, 2024):
    minmax = get_amount_of_patents(year, get_token())
    if minmax is not None and minmax[0] is not None and minmax[1] is not None:
        min_value = minmax[0]
        max_value = minmax[1]


        new_row = pd.DataFrame({'year': [year], 'min': [min_value], 'max': [max_value], 'total': [max_value - min_value + 1]})
        df = pd.concat([df, new_row], ignore_index=True)
        print(f"Year: {year}, Min: {min_value}, Max: {max_value}, Total: {max_value - min_value + 1}")
    else:
        print(f"Failed to retrieve data for year: {year}")
    time.sleep(3)

Failed to retrieve data for year: 1772
Failed to retrieve data for year: 1773
Failed to retrieve data for year: 1774
Failed to retrieve data for year: 1775
Failed to retrieve data for year: 1776
Failed to retrieve data for year: 1777
Failed to retrieve data for year: 1778
Failed to retrieve data for year: 1779
Failed to retrieve data for year: 1780
Failed to retrieve data for year: 1781
Failed to retrieve data for year: 1782
Failed to retrieve data for year: 1783
Failed to retrieve data for year: 1784
Failed to retrieve data for year: 1785
Failed to retrieve data for year: 1786
Year: 1787, Min: 178701616, Max: 178701616, Total: 1
Failed to retrieve data for year: 1788
Failed to retrieve data for year: 1789
Failed to retrieve data for year: 1790
Failed to retrieve data for year: 1791
Failed to retrieve data for year: 1792
Failed to retrieve data for year: 1793
Failed to retrieve data for year: 1794
Failed to retrieve data for year: 1795
Failed to retrieve data for year: 1796
Failed to r

In [12]:
print(df)

     year          min          max        total
0    1784    178401432    178401432            1
1    1799    179902340    179902340            1
2    1817    181704136    181704136            1
3    1787    178701616    178701616            1
4    1830           53           61            9
..    ...          ...          ...          ...
101  2003         6941  32002000572  32001993632
102  2005        99961       113468        13508
103  2006       106080       115084         9005
104  2010  32007000305  32010000491      3000187
105  2012  22008000557  32012001077  10004000521

[106 rows x 4 columns]


In [13]:
df.to_csv('estimation.csv', index=False)
